# analysis for getting evidance

In [ ]:
import time
import pathlib
from os.path import isfile

import math
import torch
import numpy as np

import models
from utils import *
from data import DataLoader


class config(object):
    def __init__(self):
        self.dataset = 'cifar10'
        self.arch = 'resnet'
        self.layers = 14
        self.ckpt = 'ckpt_best.pth'
        self.bn = False
        self.width_mult = 1.0
        self.cuda = False
        self.type = 'max'

In [ ]:
def main():
    global opt, arch_name
    opt = config()

    # set model name
    arch_name = set_arch_name(opt)

    print('\n=> creating model \'{}\''.format(arch_name))
    model = models.__dict__[opt.arch](data=opt.dataset, num_layers=opt.layers,
                                      width_mult=opt.width_mult, batch_norm=opt.bn)

    if model is None:
        print('==> unavailable model parameters!! exit...\n')
        exit()

    # checkpoint file
    ckpt_dir = pathlib.Path('checkpoint')
    dir_path = ckpt_dir / arch_name / opt.dataset
    ckpt_file = dir_path / opt.ckpt

    if isfile(ckpt_file):
        print('==> Loading Checkpoint \'{}\''.format(opt.ckpt))
        checkpoint = load_model(model, ckpt_file,
                                main_gpu=None, use_cuda=False)
        print('===> Loaded Checkpoint \'{}\' (epoch {})'.format(
            opt.ckpt, checkpoint['epoch']))
        print('\n==> Get and Calculate distribution of absolute pcc')
        mat_dist = get_dist_abs_pcc(model, opt.type)
        print('\n===> done')
        return mat_dist
    else:
        print('==> no Checkpoint found at \'{}\''.format(
            opt.ckpt))
        return


def get_dist_abs_pcc(model, type='max'):
    w_kernel = get_kernel(model, opt)
    num_layer = len(w_kernel)

    dist_all = []
    for i in tqdm(range(num_layer), ncols=80, unit='layer'):
        ref_layer = torch.Tensor(w_kernel[i])
        if opt.arch in hasDiffLayersArchs:
            ref_layer = ref_layer.view(-1, 9)
        else:
            ref_layer = ref_layer.view(len(w_kernel[i]), -1)

        ref_length = ref_layer.size()[0]

        ref_mean = ref_layer.mean(dim=1, keepdim=True)
        ref_norm = ref_layer - ref_mean
        ref_norm_sq = (ref_norm * ref_norm).sum(dim=1)
        ref_norm_sq_rt = torch.sqrt(ref_norm_sq)

        dist = []
        for j in range(i+1, num_layer):
            cur_weight = torch.Tensor(w_kernel[j])

            # change kernels to dw-kernel
            if opt.arch in hasDiffLayersArchs:
                cur_weight = cur_weight.view(-1, 9)
            else:
                cur_weight = cur_weight.view(len(w_kernel[j]), -1)

            cur_length = cur_weight.size()[0]

            cur_mean = cur_weight.mean(dim=1, keepdim=True)
            cur_norm = cur_weight - cur_mean
            cur_norm_sq_rt = torch.sqrt((cur_norm * cur_norm).sum(dim=1))

            cur_dist = []
            for k in range(cur_length):
                numer = torch.matmul(cur_norm[k], ref_norm.T)
                denom = ref_norm_sq_rt * cur_norm_sq_rt[k]
                pcc = numer / denom
                abs_pcc = torch.abs(pcc)
                if type == 'max':
                    cur_dist.append(torch.max(abs_pcc).item())
                elif type == 'min':
                    cur_dist.append(torch.min(abs_pcc).item())
                elif type == 'avg':
                    cur_dist.append(torch.mean(abs_pcc).item())
                elif type == 'med':
                    cur_dist.append(torch.median(abs_pcc).item())
            dist.append(cur_dist)
        dist_all.append(dist)
    return dist_all


if __name__ == '__main__':
    start_time = time.time()
    arr_dist = main()
    elapsed_time = time.time() - start_time
    print("====> total time: {:.2f}s".format(elapsed_time))


In [ ]:
#TODO: 각각에 대한 histogram 그리기
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter


# make directory
dir_plots = pathlib.Path('Histograms') / arch_name / opt.dataset / opt.type
dir_plots.mkdir(parents=True, exist_ok=True)

for i in range(len(arr_dist)):
    for j in range(len(arr_dist[i])):
        cur_dist = arr_dist[i][j]
        num_pcc = len(cur_dist)
        min_pcc = min(cur_dist)
        max_pcc = max(cur_dist)
        med_pcc = np.median(cur_dist)
        avg_pcc = np.mean(cur_dist)
        var_pcc = np.var(cur_dist)
        std_pcc = np.std(cur_dist)
        textstr = '\n'.join((
            r'$\min=%.6f$' % (min_pcc, ),
            r'$\max=%.6f$' % (max_pcc, ),
            r'$median=%.6f$' % (med_pcc, ),
            r'$\mu=%.6f$' % (avg_pcc, ),
            r'$\sigma^{2}=%.6f$' % (var_pcc, ),
            r'$\sigma=%.6f$' % (std_pcc, )))
        
        plt.style.use('seaborn-deep')
        fig, ax = plt.subplots(figsize=(8,6), dpi=150)
        cur_num = i + j + 1
        # plt.title('Histogram of {} abs PCC ref_{:02d} with cur_{:02d} layer'.format(opt.type, i, cur_num),
        #     fontsize=15, fontfamily='Times New Roman')
        y_vals, x_vals, e_ = ax.hist(cur_dist, label='PCC', bins=min(num_pcc, 256))
        y_max = round((max(y_vals) / num_pcc) + 0.02, 2)
        ax.set_yticks(ticks=np.arange(0.0, y_max * num_pcc, 0.01 * num_pcc))
        ax.set_ylim(ax.get_yticks()[0], ax.get_yticks()[-1])
        ax.set_xlim(0.0, 1.0)
        ax.yaxis.set_major_formatter(PercentFormatter(xmax=num_pcc))
        plt.legend(loc='upper right')
        # these are matplotlib.patch.Patch properties
        props = dict(boxstyle='round', facecolor='lightsteelblue', alpha=0.5)
        # place a text box in upper left in axes coords
        ax.text(0.03, 0.96, textstr, transform=ax.transAxes, fontsize=9,
                verticalalignment='top', bbox=props)
        plt.savefig(dir_plots / 'abs_pcc_ref{:02d}_cur{:02d}'.format(opt.type, i, cur_num),
                    bbox_inches='tight')
